# Rozszerzenie relacji M2M

Czasami chcemy rozszerzyć informacje o jakimś powiazaniu. Może to być np. informacja o dacie złożenia zamówienia przez klienta (to będzie o2m, ponieważ klient może mieć wiele zamówień, jedno zamówienie może być przypisane tylko do jednego klienta) lub informacja o wysokości zakonktraktowanej gaży aktora za występ w filmie (to już będzie m2m). O ile w przypadku realcji o2o i o2m takie rozszerzenie poleg na dodaniu kolejnej kolumny w tabeli, w której umieściliśmy powiązanie, o tyle w przypadku relacji m2m dodatkową kolumnę umieszczamy w tabeli pośredniej. Tworząc relacje m2m w Django nie przejmowaliśmy się tabelą pośrednią. Wiemy, że Django wygeneruje taką tabelę automatycznie. Niestety, jeżeli chcemy rozszerzyć relacje m2m o dodatkową kolumnę to sami musimy już zdefniować tabelkę pośrednią i przypisać jej odpowiednie kolumny. W tym celu tworzymy tzw. model pośredni (ang. **intermediate model**)

W przykładzie (zaczerpniętym z dokumentacji Django) mamy dwie tabele - Person i Band.

<code>
class Person(models.Model):
    name = models.CharField(max_length=128)
</code>
<code>
    def __str__(self):
        return self.name
</code>


<code>
class Group(models.Model):
    name = models.CharField(max_length=128)
</code>
<code>
    def __str__(self):
        return self.name
</code>

Osoba może wchodzić w skład wielu zespołów, a zespół może składać się z wielu osób. Dodajemy relacji m2m (po stronie zespołu).

<code>
class Person(models.Model):
    name = models.CharField(max_length=128)
</code>
<code>
    def __str__(self):
        return self.name
</code>


<code>
class Group(models.Model):
    name = models.CharField(max_length=128)
    members = models.ManyToManyField(Person)
</code>
<code>
    def __str__(self):
        return self.name
</code>

I teraz załóżmy, że dodatkowo chcemy przechować też informacje o dacie dołączenia osoby do zespołu. Taka informacja będzie dotyczyła powiązania (tzn. data dotyczyć będzie połączenia konkretnej osoby z konkretnym zespołem), a nie samej osoby lub samego zespołu. W tym celu defniujemy model pośredni, nazywamy go Membership i umieszczajamy w nim wszystkie potrzebne kolumny. Następnie za pomocą parametru `through`, w polu ManyToManyField wskazujemy, że to pole ManyToManyField realizowane jest poprzez model pośredni `Membership`.

### Pełny przykład

<code>
class Person(models.Model):
    name = models.CharField(max_length=128)
</code>
<code>
    def __str__(self):
        return self.name
</code>


<code>
class Group(models.Model):
    name = models.CharField(max_length=128)
    members = models.ManyToManyField(Person, , through='Membership')
</code>
<code>
    def __str__(self):
        return self.name
</code>

<code>
class Membership(models.Model):
    person = models.ForeignKey(Person, on_delete=models.CASCADE)
    group = models.ForeignKey(Group, on_delete=models.CASCADE)
    date_joined = models.DateField()
</code>

Jeżeli nie masz powyższego przykładu w swoim projekcie, przeklej go (do pliku models.py wybranej aplikacji), a następnie przygotuj i wykonaj migracje. W dalszej części notatnika będziemy korzystać ze wszystkich trzech modeli.

Standradowo, najpierw linijki, które pozwolą nam swobodnie korzystać z Django w notatniku Jupyter.

In [ ]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

Importujemy potrzebne biblioteki i modele

In [ ]:
from datetime import date

from relations_app.models import Person, Band, Membership

**C z CRUD**

In [ ]:
# Tworzymy wpisy Ringo Starr, John Lennon i Paul McCartney w tabeli Person i wpis The Beatles w tabeli Band
ringo = Person.objects.create(name="Ringo Starr")
john = Person.objects.create(name="John Lennon")
paul = Person.objects.create(name="Paul McCartney")

beatles = Band.objects.create(name="The Beatles")

Popatrzmy na utwrzone obiekty

In [ ]:
print(dir(beatles))  # widzimy menadżer powiązany - members

In [ ]:
print(dir(paul))  # widzimy menadżer powiązany - band_set

W obu modelach poza poznanym już wcześniej menadżerem powiązanym mamy też menadżer powiązany do modelu pośredniego - membership_set

Dodajemy powiązanie z zespołem

In [ ]:
# Version I (bezpośrednie użycie modelu pośredniego)
m1 = Membership(person=ringo, band=beatles, date_joined=date(1962, 8, 16))
m1.save()

In [ ]:
# Version II (użycie menadżera powiązanego modelu Band)
beatles.members.add(john, through_defaults={'date_joined': date(1960, 8, 1)})

# zwróć uwagę, że dane dla kolumn rozszerzających podajemy w postaci słownika przy użyciu parametru through_defaults, 
# w słowniku muszą znaleźć się wszystkie rozszerzające model pośredni dane.

In [ ]:
# Version III (użycie menadżera powiązanego modelu Person)
paul.band_set.add(beatles, through_defaults={'date_joined': date(1960, 8, 1)})

In [ ]:
# Version IV (za pomocą metody create - jednoczesne utworzenie wpisu i dodanie powiązania)
george = beatles.members.create(name="George Harrison", through_defaults={'date_joined': date(1960, 8, 1)})

# tutaj w parametrach trzeba podać wszystkie dane potrzebne do utworzenia wpisu w tabeli Person

In [ ]:
# Version V (za pomocą metody set - analogicznie jak w przypadk 'czystej' relacji m2m)
beatles.members.set([john, paul, ringo], through_defaults={'date_joined': date(1960, 8, 1)})

# metody set możemy użyć jeżeli wszystkie podane w liście wpisy mają te same wartości w kolumnach rozszerzających 
# model pośredni (czylie te same wartości w słowniku przypisywanym do parametru through_defaults).

**R z CRUD**

Znajdź wszystkie zespoły, które w składzie posiadają osoby z wartością w kolumnie name zaczynającą się na 'Paul'

In [ ]:
Band.objects.filter(members__name__startswith='Paul')

Znajdź wszystkie osoby, które przyłączyły się do zespołu 'The Beatles' po 1 Jan 1961

In [ ]:
Person.objects.filter(band__name='The Beatles', membership__date_joined__gt=date(1961, 1, 1))

A jak zapytać się o informacje uzupełniającą (w naszym przypadku o datę przyłączenia się do zespołu)? W którym roku do zespołu 'The Beatles' przyłączył się Ringo Starr?

In [ ]:
# Version I (przez bezpośrednie odpytanie modelu pośredniego)
ringos_membership = Membership.objects.get(band=beatles, person=ringo)
ringos_membership.date_joined

In [ ]:
# Version II (za pomocą menadżera powiązanego z modelem pośrednim z poziomu modelu Person)
ringos_membership = ringo.membership_set.get(band=beatles)
ringos_membership.date_joined

In [ ]:
# Version III (za pomocą menadżera powiązanego z modelem pośrednim z poziomu modelu Band)
ringos_membership = beatles.membership_set.get(person=ringo)
ringos_membership.date_joined

**U z CRUD**

**D z CRUD**

W jaki sposób możemy usuwać powiązania?

In [ ]:
# Version I (bezpośrednio z modelu pośredniego)
ringos_membership = Membership.objects.get(band=beatles, person=ringo)
ringos_membership.delete()

In [ ]:
# Version II (za pomocą metody remove menadżera powiązanego modelu Band)
beatles.members.remove(john)

# metoda remove zwraca None

In [ ]:
# Version III (za pomocą metody remove menadżera powiązanego modelu Person)
paul.band_set.remove(beatles)

# Zagadnienia dodatkowe

**Optymlizcja zapytań**


W jaki sposób, za pomocą dotychczas poznanych narzędzi moglibyś zrealizować joina?

Weźmy tabelki Framework i language

In [ ]:
from relations_app.models import Framework, Language

Z tabelki Framework wyciągnijmy Django

In [ ]:
django_qs = Framework.objects.filter(name='django')

# na bazie wykona się taka sqlka
print(django_qs.query)

In [ ]:
django = django_qs[0]
django

In [ ]:
# jeżeli teraz chcemy wyciągnąć powiązany z wpisem django wpis z tabelki Language
python = django.language

# to django orm musi wykonać kolejne zapytanie na bazę
python

Załóżmy teraz, że chcemy wyświetlić język wszystkich frameworków zapisanych w bazie. Wykorzystując dotychczas poznane narzędzie kod będzie wyglądał mniej więcej tak:

In [ ]:
frameworks = Framework.objects.all()

for framework in frameworks:
    print(framework.language)

Przy założeniu, że w bazie mamy zapisane 100 frameworków django będzie musiał uderzyć na bazę 101 razy (raz, żeby wyciągnąć wszystkie frameworki i później po razie na każdy framework, żeby wyciągnąć jego język). 

Znając klauzulę JOIN wiemy, że z jej użyciem możnabyłoby wszystkie potrzebne informacje wyciągnąc za pomocą jednego zapytania na bazę. Ma to szczególne znaczenie, kiedy wyciągamy dużo wpisów (np. 100 albo 1000). 1000 dodatkowych zapytań do bazy może wygenerować już zauważalne wydłużenie czasu wykonywania kodu. Jak zmusić django orm do wykonania JOINa?

Za pomocą django orm **instrukcje join** możemy zrealizować na dwa sposoby:
- `select_related`
- `prefetch_related`

To, której metody będziemy chcieli użyć zależy od charaketru relacji wzdłuż, której złączamy tabelki.

## select_related

`select_related` wykonuje trdycyjnego joina, bez żadnych dodatkowych usprawnień. Używamy w przypadku relacji o2o oraz o2m. Proba użycia na relacji m2m zakończy się błędem z przyczyn, o których poniżej.

In [ ]:
frameworks = Framework.objects.select_related()

# tym razem na bazie wykona się taka sqlka
print(frameworks.query)

Widzimy joina. Django orm wyciągnie z bazy informacje o wpisach powiązanych i powkłada te informacje do odpowiednich obiektów w taki sposób, że odwołując się do języka frameworka, orm nie będzie potrzebował, ponownie uderzyć na bazę.

In [ ]:
# tym razem ten kod wykona się bez uderzania na bazę
for framework in frameworks:
    print(framework.language)

Jeżeli chcemy zrobić joina tylko z wybranymi powiązaniami (a nie ze wszystkimi), dopisujemy nazwy wybranych kluczy obcy jako parametry pozycyjne metody select_related.

In [ ]:
frameworks = Framework.objects.select_related('language')
print(frameworks.query)  
# w tym przykładzie nie widzimy różnicy pomiędzy tym zapytaniem, a poprzednim ponieważ tabelka framework ma tylko 
# jedno powiązanie (z tabelką Language)

Jeżeli chcielibyśmy robić kolejnego joina z wykorzystaniem danych z tabeli zjoinowanej, możemy posłużyć się `field lookupami`. Założmy hipotetycznie, że tabelka language ma jeszcze kolumnę type, która jest kluczem obcym do tabelki Type (niech tabelka Type reprezentuje typ języka programowania, np. statyczny, dynamiczny). Jeżeli chcielibyśmy zjoinować również tą kolejną tabelkę, możemy to zrobić w ten sposób:

In [ ]:
# frameworks = Framework.objects.select_related('language__type')

# W ten sposób możemy łańcuchować lookupy joinując dalsze tabelki

W wyniku wykonywania joinów czasami (zwłaszcza w przypadku relacji m2m) możemy wyciągać z bazy dużą ilość danych. Z myślą o takich przypadkach django orm udostępnia drugą metodę na zrealizowanie joina - `prefetch_related`.

### prefetch_related

In [ ]:
frameworks = Framework.objects.prefetch_related()

print(frameworks.query)

Tutaj nie widzimy, żeby django zrobił jakiegoś joina, ale pod spodem django wykonał dwa zapytania na bazę. Jedno to to, którego sql widzimy powyżej. Drugie zapytanie było o wpisy powiązane. A następnie połączył te informacje ze sobą już po stronie pythona. W efekcie zamiast 101 zapytań zrobiliśmy dwa. 

`prefech_related` używamy kiedy nie chcemy uderzać wiele razy na bazę, a jednocześnie wiemy że w wyniku pojdynczego zapytania dostaniemy ogromną liczbę danych, których przetworzenie może zająć więcej czasu niż przetworzenie tej samej ilości danych, ale w dwóch mniejszych zbiorach.

Powyższych reguł nie jesteśmy w stanie zaobserwować bez użycia dodatkowych narzędzi. Jednym z najlepszych sposobów na śledzenie jakości zapytań bazodanowych po stronie Django jest użycie Django Debug Toolbar (https://django-debug-toolbar.readthedocs.io/en/latest/installation.html), dodatku do django, który monitoruje wiele parametrów widoku, wśród których znajduje się liczba zapytań sql i czas ich wykonania.